# XGBRegression (P,A,D coordinates to LMA Features)

In [3]:
# Current best:
#datasets/pad/Fs_B_S_DANCE_WALK_0.5sec.csv
"""
NOTE TO SELF: DON'T FREAKING USE THE STANDARDIZED/NORMALIZED DATASETS HERE, THEY WON'T WORK. YOU'RE STANDARDIZING THE OUTPUT DUMB DUMB!
                
"""

'\n\n                \n'

## Imports

In [21]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.lines as mlines

import tensorflow as tf

import sklearn
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold

import xgboost as xgb

xgb.set_config(verbosity=0)

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import normalize

from sklearn.utils import shuffle
import math
import joblib


## Load Data

In [43]:
dataset = pd.read_csv('datasets/Fs_B_DANCE_WALK_KIN_1sec.csv')

### Split into Test and Train data

In [44]:
train_dataset = dataset.sample(frac=0.8, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

print("No Training Samples:",train_dataset.shape[0])
print("No Test Samples:",test_dataset.shape[0])

train_dataset = shuffle(train_dataset)
test_dataset = shuffle(test_dataset)

No Training Samples: 9660
No Test Samples: 2415


### Split Features from Targets

In [45]:
train_y_sets = []
test_y_sets = []
# List of np.arrays each containing the LMA features of a given column (i.e one list per feature)

In [46]:
train_X = pd.concat([train_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
test_X = pd.concat([test_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)

In [47]:
for colname in train_dataset.columns:
    train_y_sets.append(pd.concat([train_dataset.pop(x) for x in [colname]], axis=1))
    
for colname in test_dataset.columns:
    test_y_sets.append(pd.concat([test_dataset.pop(x) for x in [colname]], axis=1))

In [48]:
train_X.head()

,EMOTION_P,EMOTION_A,EMOTION_D
11388,-0.85,-0.1,-0.8
903,-0.50,0.6,0.9
52,-0.50,0.6,0.9
8043,0.10,-0.7,-0.2
7188,-0.50,0.6,0.9


In [49]:
for i in range(len(train_y_sets)):
    print(train_y_sets[i].columns[0])

max_hand_distance
avg_l_hand_hip_distance
avg_r_hand_hip_distance
max_stride_length
avg_l_hand_chest_distance
avg_r_hand_chest_distance
avg_l_elbow_hip_distance
avg_r_elbow_hip_distance
avg_chest_pelvis_distance
avg_neck_chest_distance
avg_neck_rotation_w
avg_neck_rotation_x
avg_neck_rotation_y
avg_neck_rotation_z
avg_total_body_volume
avg_triangle_area_hands_neck
avg_triangle_area_feet_hips
l_hand_speed
r_hand_speed
l_foot_speed
r_foot_speed
neck_speed
l_hand_acceleration_magnitude
r_hand_acceleration_magnitude
l_foot_acceleration_magnitude
r_foot_acceleration_magnitude
neck_acceleration_magnitude


## Define Model

In [50]:
xgbr = xgb.XGBRegressor(verbosity=1) 

params = {
        'eta': [0.01, 0.05, 0.1],
        'min_child_weight': [1, 5, 11, 21],
        'max_depth': [3, 6, 10, 15],
        'gamma': [0, 0.001, 0.01],
        'subsample': [0.75, 1],
        'colsample_bytree': [0.75, 1],
        'lambda': [1, 1.25],
        'alpha': [0.0, 0.25]
        }

models = []

for i in range(len(train_y_sets)):
    models.append(
        xgb.XGBRegressor(
                    n_estimators=1500, learning_rate=0.05, max_depth=6, min_child_weight=11, 
                    reg_alpha=0.25, reg_lambda=1.25, gamma=0.01,
                    subsample=0.75, colsample_bytree=0.75, objective="reg:squarederror",
                    tree_method='gpu_hist'
                )
    )

## Train Model

In [51]:
for i in range(len(models)):
    print(train_y_sets[i].columns[0])
    models[i].fit(train_X, train_y_sets[i])

max_hand_distance
avg_l_hand_hip_distance
avg_r_hand_hip_distance
max_stride_length
avg_l_hand_chest_distance
avg_r_hand_chest_distance
avg_l_elbow_hip_distance
avg_r_elbow_hip_distance
avg_chest_pelvis_distance
avg_neck_chest_distance
avg_neck_rotation_w
avg_neck_rotation_x
avg_neck_rotation_y
avg_neck_rotation_z
avg_total_body_volume
avg_triangle_area_hands_neck
avg_triangle_area_feet_hips
l_hand_speed
r_hand_speed
l_foot_speed
r_foot_speed
neck_speed
l_hand_acceleration_magnitude
r_hand_acceleration_magnitude
l_foot_acceleration_magnitude
r_foot_acceleration_magnitude
neck_acceleration_magnitude


## Test Model

### Training Score

In [52]:
for i in range(len(models)):
    score = models[i].score(train_X, train_y_sets[i])
    print(train_y_sets[i].columns[0])
    print("Training score: ", score)

    print()

max_hand_distance
Training score:  0.19544739718587123

avg_l_hand_hip_distance
Training score:  0.12159866542503805

avg_r_hand_hip_distance
Training score:  0.09243928938053869

max_stride_length
Training score:  0.19395649232062984

avg_l_hand_chest_distance
Training score:  0.09277452654304086

avg_r_hand_chest_distance
Training score:  0.057797524122139565

avg_l_elbow_hip_distance
Training score:  0.09714782196739469

avg_r_elbow_hip_distance
Training score:  0.06046742101653979

avg_chest_pelvis_distance
Training score:  -3461316277.1312356

avg_neck_chest_distance
Training score:  -1.3056542069733723e-07

avg_neck_rotation_w
Training score:  0.015630413654506126

avg_neck_rotation_x
Training score:  0.07097370977321982

avg_neck_rotation_y
Training score:  0.02034396900863711

avg_neck_rotation_z
Training score:  0.1463904688406813

avg_total_body_volume
Training score:  0.15972920680310654

avg_triangle_area_hands_neck
Training score:  0.043844132477119935

avg_triangle_area_f

### Test Set MAE & MSE

In [53]:
for i in range(len(models)):
    pred_y = models[i].predict(test_X)
    mse = mean_squared_error(test_y_sets[i], pred_y)
    mae = mean_absolute_error(test_y_sets[i], pred_y)
    print(test_y_sets[i].columns[0])
    print("MSE: %.2f" % mse)
    print("MAE: %.2f" % mae)
    print("Example: ", test_y_sets[i].iloc[0][0])
    print()

max_hand_distance
MSE: 0.07
MAE: 0.22
Example:  0.2499922076973424

avg_l_hand_hip_distance
MSE: 0.02
MAE: 0.12
Example:  0.2178728496732131

avg_r_hand_hip_distance
MSE: 0.03
MAE: 0.13
Example:  0.080778272430834

max_stride_length
MSE: 0.03
MAE: 0.13
Example:  0.3359630561132576

avg_l_hand_chest_distance
MSE: 0.01
MAE: 0.08
Example:  0.100752431206083

avg_r_hand_chest_distance
MSE: 0.01
MAE: 0.08
Example:  0.2746658878325115

avg_l_elbow_hip_distance
MSE: 0.01
MAE: 0.05
Example:  0.2158367520240179

avg_r_elbow_hip_distance
MSE: 0.01
MAE: 0.06
Example:  0.2346567291208245

avg_chest_pelvis_distance
MSE: 0.00
MAE: 0.00
Example:  0.2861510001383717

avg_neck_chest_distance
MSE: 0.00
MAE: 0.00
Example:  0.2788084298437885

avg_neck_rotation_w
MSE: 0.01
MAE: 0.07
Example:  -0.2265473076656396

avg_neck_rotation_x
MSE: 0.09
MAE: 0.21
Example:  0.0585464164441948

avg_neck_rotation_y
MSE: 0.02
MAE: 0.10
Example:  -0.5676475833273876

avg_neck_rotation_z
MSE: 0.05
MAE: 0.13
Example:  0.78

### Prediction Examples

In [54]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("../../emotion_classifier/model_training/models/l2p_dance_model.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("../../emotion_classifier/model_training/models/l2a_dance_model.json")

model_d = xgb.XGBRegressor(verbosity=0)
model_d.load_model("../../emotion_classifier/model_training/models/l2d_dance_model.json")

scaler = joblib.load('../../emotion_classifier/model_training/datasets/scalers/standardizers/Fs_B_S_DANCE_WALK_KIN_0.5sec.pkl') 

In [56]:
index = 0

real_coordinates = np.asarray([test_X.iloc[index]])

generated_features = []
for i in range(len(models)):
    generated_features.append(models[i].predict(real_coordinates)[0])
    #print(test_y_sets[i].columns[0], " - ", generated_features[i])
    
generated_features = pd.DataFrame([generated_features], columns=[
            "max_hand_distance",
            "avg_l_hand_hip_distance",
            "avg_r_hand_hip_distance",
            "max_stride_length",
            "avg_l_hand_chest_distance",
            "avg_r_hand_chest_distance",
            "avg_l_elbow_hip_distance",
            "avg_r_elbow_hip_distance",
            "avg_chest_pelvis_distance",
            "avg_neck_chest_distance",
            "avg_neck_rotation_w", "avg_neck_rotation_x", "avg_neck_rotation_y", "avg_neck_rotation_z",
            "avg_total_body_volume",
            "avg_triangle_area_hands_neck",
            "avg_triangle_area_feet_hips",
          
            "l_hand_speed",
            "r_hand_speed",
            "l_foot_speed",
            "r_foot_speed",
            "neck_speed",
          
            "l_hand_acceleration_magnitude",
            "r_hand_acceleration_magnitude",
            "l_foot_acceleration_magnitude",
            "r_foot_acceleration_magnitude",
            "neck_acceleration_magnitude",
         ])

generated_features = scaler.transform(generated_features)

y_p = model_p.predict(generated_features)
y_a = model_a.predict(generated_features)
y_d = model_d.predict(generated_features)

print('Real: %s' % np.asarray(real_coordinates))
print('Predicted: %s' % [y_p[0], y_a[0], y_d[0]])

Real: [[-0.35  0.7  -0.8 ]]
Predicted: [0.027638767, -0.1088836, -0.036087155]


## Save Models

In [118]:
# save to JSON
model_p.save_model("models/l2p_dance_model.json")
model_a.save_model("models/l2a_dance_model.json")
model_d.save_model("models/l2d_dance_model.json")

## Load Models

In [856]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("models/l2p_dance_model_v2.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("models/l2a_dance_model_v2.json")

# Hyperparameter Grid Search with XGBoost

In [2]:
from datetime import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf

import sklearn
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error


import xgboost as xgb

xgb.set_config(verbosity=2)

from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
import joblib

## Load Data

In [3]:
dataset = pd.read_csv('datasets/Fs_B_DANCE_WALK_KIN_0.5sec.csv')

### Split into Test and Train data

In [4]:
train_dataset = dataset.sample(frac=0.8, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

print("No Training Samples:",train_dataset.shape[0])
print("No Test Samples:",test_dataset.shape[0])

train_dataset = shuffle(train_dataset)
test_dataset = shuffle(test_dataset)

No Training Samples: 39539
No Test Samples: 9885


### Split Features from Targets

In [5]:
train_y_sets = []
test_y_sets = []
# List of np.arrays each containing the LMA features of a given column (i.e one list per feature)

In [6]:
train_X = pd.concat([train_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
test_X = pd.concat([test_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)

In [7]:
for colname in train_dataset.columns:
    train_y_sets.append(pd.concat([train_dataset.pop(x) for x in [colname]], axis=1))
    
for colname in test_dataset.columns:
    test_y_sets.append(pd.concat([test_dataset.pop(x) for x in [colname]], axis=1))

In [8]:
train_X.head()

,EMOTION_P,EMOTION_A,EMOTION_D
7012,-0.85,-0.10,-0.80
19814,0.90,-0.25,0.65
423,-0.40,0.25,-0.10
45807,-0.50,0.60,0.90
28216,-0.35,0.70,-0.80


In [9]:
for i in range(len(train_y_sets)):
    print(train_y_sets[i].columns[0])

max_hand_distance
avg_l_hand_hip_distance
avg_r_hand_hip_distance
max_stride_length
avg_l_hand_chest_distance
avg_r_hand_chest_distance
avg_l_elbow_hip_distance
avg_r_elbow_hip_distance
avg_chest_pelvis_distance
avg_neck_chest_distance
avg_neck_rotation_w
avg_neck_rotation_x
avg_neck_rotation_y
avg_neck_rotation_z
avg_total_body_volume
avg_triangle_area_hands_neck
avg_triangle_area_feet_hips
l_hand_speed
r_hand_speed
l_foot_speed
r_foot_speed
neck_speed
l_hand_acceleration_magnitude
r_hand_acceleration_magnitude
l_foot_acceleration_magnitude
r_foot_acceleration_magnitude
neck_acceleration_magnitude


## RandomSearchCV

In [10]:
# A parameter grid for XGBoost
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
params = {
        'eta': [0.01, 0.05, 0.1],
        'min_child_weight': [1, 5, 11, 21],
        'max_depth': [3, 6, 10, 15],
        'gamma': [0, 0.001, 0.01],
        'subsample': [0.75, 1],
        'colsample_bytree': [0.75, 1],
        'lambda': [1, 1.25],
        'alpha': [0.0, 0.25]
        }

n_iter = 200

In [11]:
models = []

for i in range(len(train_y_sets)):
    models.append(
        xgb.XGBRegressor(
                    n_estimators=1500, 
                    objective="reg:squarederror",
                    tree_method='gpu_hist'
                )
    )

In [ ]:
# Pleasure
# run randomized search

random_searches = []

for i in range(len(models)):
    print(train_y_sets[i].columns[0])
    
    kfold = KFold(n_splits=3, shuffle=True)

    random_search = RandomizedSearchCV(models[i], param_distributions=params,
                               cv=kfold, scoring='neg_mean_squared_error', n_iter = n_iter)

    start = time.time()
    random_search.fit(train_X, train_y_sets[i])
    
    random_searches.append(random_search)

    print("GridSearchCV took %.2f seconds"
          " parameter settings." % ((time.time() - start)))
    
    print()

max_hand_distance
GridSearchCV took 1006.77 seconds parameter settings.

avg_l_hand_hip_distance
GridSearchCV took 1297.47 seconds parameter settings.

avg_r_hand_hip_distance


### Results

In [20]:
best_regressors = []

for i in range(len(models)):
    print(train_y_sets[i].columns[0])
    best_regressor = random_searches[i].best_estimator_

    print(best_regressor.get_params())
    best_regressors.append(best_regressor)
    print()

max_hand_distance
{'objective': 'reg:squarederror', 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 0.75, 'enable_categorical': False, 'gamma': 0.01, 'gpu_id': 0, 'importance_type': None, 'interaction_constraints': '', 'learning_rate': 0.100000001, 'max_delta_step': 0, 'max_depth': 3, 'min_child_weight': 11, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 1500, 'n_jobs': 12, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 0.75, 'tree_method': 'gpu_hist', 'validate_parameters': 1, 'verbosity': None, 'lambda': 1, 'eta': 0.1, 'alpha': 0.0}

avg_l_hand_hip_distance
{'objective': 'reg:squarederror', 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 0.75, 'enable_categorical': False, 'gamma': 0.01, 'gpu_id': 0, 'importance_type': None, 'interaction_constraints': '', 'learning_rate

In [17]:
for i in range(len(best_regressors)):
    pred_y = best_regressors[i].predict(test_X)
    mse = mean_squared_error(test_y_sets[i], pred_y)
    mae = mean_absolute_error(test_y_sets[i], pred_y)
    print(test_y_sets[i].columns[0])
    print("MSE: %.2f" % mse)
    print("MAE: %.2f" % mae)
    print("Example: ", test_y_sets[i].iloc[0][0])
    print()

max_hand_distance
MSE: 0.07
MAE: 0.21
Example:  0.3589408342450211

avg_l_hand_hip_distance
MSE: 0.03
MAE: 0.12
Example:  0.2153851628650174

avg_r_hand_hip_distance
MSE: 0.03
MAE: 0.13
Example:  0.2099208139313444

max_stride_length
MSE: 0.03
MAE: 0.13
Example:  0.3014104931041282

avg_l_hand_chest_distance
MSE: 0.01
MAE: 0.09
Example:  0.437371900332585

avg_r_hand_chest_distance
MSE: 0.01
MAE: 0.09
Example:  0.4379690956765778

avg_l_elbow_hip_distance
MSE: 0.01
MAE: 0.06
Example:  0.315309309619572

avg_r_elbow_hip_distance
MSE: 0.01
MAE: 0.06
Example:  0.315764635359871

avg_chest_pelvis_distance
MSE: 0.00
MAE: 0.00
Example:  0.2861510014037163

avg_neck_chest_distance
MSE: 0.00
MAE: 0.00
Example:  0.2785828270734098

avg_neck_rotation_w
MSE: 0.01
MAE: 0.07
Example:  -0.0303669968027351

avg_neck_rotation_x
MSE: 0.10
MAE: 0.21
Example:  0.9944506264704334

avg_neck_rotation_y
MSE: 0.02
MAE: 0.10
Example:  -0.0616060786573984

avg_neck_rotation_z
MSE: 0.06
MAE: 0.13
Example:  -0.04

In [18]:
# save to JSON
for i in range(len(best_regressors)):
    best_regressors[i].save_model("models/0.5_sec/" + test_y_sets[i].columns[0] +".json")

### Predictions

In [46]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("../../emotion_classifier/model_training/models/l2p_dance_model.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("../../emotion_classifier/model_training/models/l2a_dance_model.json")

model_d = xgb.XGBRegressor(verbosity=0)
model_d.load_model("../../emotion_classifier/model_training/models/l2d_dance_model.json")

scaler = joblib.load('../../emotion_classifier/model_training/datasets/scalers/standardizers/Fs_B_S_DANCE_WALK_KIN_0.5sec.pkl') 

In [47]:
index = 10

real_coordinates = np.asarray([test_X.iloc[index]])

generated_features = []
for i in range(len(models)):
    generated_features.append(best_regressors[i].predict(real_coordinates)[0])
    #print(test_y_sets[i].columns[0], " - ", generated_features[i])
    
generated_features = pd.DataFrame([generated_features], columns=[
            "max_hand_distance",
            "avg_l_hand_hip_distance",
            "avg_r_hand_hip_distance",
            "max_stride_length",
            "avg_l_hand_chest_distance",
            "avg_r_hand_chest_distance",
            "avg_l_elbow_hip_distance",
            "avg_r_elbow_hip_distance",
            "avg_chest_pelvis_distance",
            "avg_neck_chest_distance",
            "avg_neck_rotation_w", "avg_neck_rotation_x", "avg_neck_rotation_y", "avg_neck_rotation_z",
            "avg_total_body_volume",
            "avg_triangle_area_hands_neck",
            "avg_triangle_area_feet_hips",
          
            "l_hand_speed",
            "r_hand_speed",
            "l_foot_speed",
            "r_foot_speed",
            "neck_speed",
          
            "l_hand_acceleration_magnitude",
            "r_hand_acceleration_magnitude",
            "l_foot_acceleration_magnitude",
            "r_foot_acceleration_magnitude",
            "neck_acceleration_magnitude",
         ])

generated_features = scaler.transform(generated_features)

y_p = model_p.predict(generated_features)
y_a = model_a.predict(generated_features)
y_d = model_d.predict(generated_features)

print('Real: %s' % np.asarray(real_coordinates))
print('Predicted: %s' % [y_p[0], y_a[0], y_d[0]])

Real: [[ 0.9  -0.25  0.65]]
Predicted: [0.45704892, 0.14212963, 0.21817927]
